In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from traffic_signaling.city_plan import *

INPUT_FOLDER = 'traffic_signaling/input_data'
DATA = [
    'a.txt',
    'b.txt',
    'c.txt',
    'd.txt',
    'e.txt',
    'f.txt'
]

In [2]:
def path_duration(path):
    # The car path begins at the end of the first street
    return sum(street.length for street in path[1:])

def car_summary(plan):
    streets = np.array(plan.streets)
    path_durations = np.array([path_duration(streets[car.path]) for car in plan.cars])
    paths_in_time = path_durations[path_durations <= plan.duration]

    print('Cars that can finish in time: '
          f'{len(paths_in_time)}/{len(plan.cars)} '
          f'({100 * len(paths_in_time) / len(plan.cars):.2f}%)')


In [3]:
def intersection_summary(plan):
    intersections_used = len([i for i in plan.intersections if i.used])

    # Intersections with non-trivial schedule
    intersections_with_schedule = len([i for i in plan.intersections if i.non_trivial])
    intersections_one_street_used = intersections_used - intersections_with_schedule

    print('Intersections used: '
          f'{intersections_used}/{len(plan.intersections)} '
          f'({100 * intersections_used / len(plan.intersections):.2f}%)')
    print('Intersections with exactly one street used: '
          f'{intersections_one_street_used}/{len(plan.intersections)} '
          f'({100 * intersections_one_street_used / len(plan.intersections):.2f}%)')
    print('Intersections with non-trivial schedule: '
          f'{intersections_with_schedule}/{len(plan.intersections)} '
          f'({100 * intersections_with_schedule / len(plan.intersections):.2f}%)')


In [4]:
def street_summary(plan):
    streets_used = [street for street in plan.streets if street.used]
    print('Used streets: '
          f'{len(streets_used)}/{len(plan.streets)} '
          f'({100 * len(streets_used) / len(plan.streets):.2f}%)')

In [5]:
def upper_bound(plan):
    """
    Theoretical maximum score if none of the cars ever has to wait at a traffic light.
    """
    streets = np.array(plan.streets)
    path_durations = np.array([path_duration(streets[car.path]) for car in plan.cars])
    score = np.sum(plan.duration - path_durations + plan.bonus)
    print(f'Upper bound of the score: {score:,}')
    return score

In [6]:
def histogram(plan):
    # used streets from intersections with non-trivial schedule
    streets = plan.streets
    street_ids = [streets[s].id for i in plan.intersections if i.non_trivial for s in i.incoming if streets[s].used]
    car_counts = np.zeros(len(plan.streets))
    for car in plan.cars:
        car_counts[car.path[:-1]] += 1

    #np.histogram(car_counts[street_ids])
    values, counts = np.unique(car_counts[street_ids], return_counts=True)
    values = values.astype(int)
    print(f'Values: {values}')
    print(f'Counts: {counts}')

    plt.bar(values, counts)
    plt.title('Histogram of cars passing through street')
    plt.xlabel('Cars passing through')
    plt.ylabel('Number of streets')

    plt.show()


In [7]:
upper_bounds = []
for d in DATA:
    plan = CityPlan(os.path.join(INPUT_FOLDER, d))
    print(d)
    intersection_summary(plan)
    street_summary(plan)
    car_summary(plan)
    upper_bounds.append(upper_bound(plan))
    print('-' * 70)
    #histogram(plan)

print(f'TOTAL UPPER BOUND: {sum(upper_bounds):,}')

a.txt
Intersections used: 3/4 (75.00%)
Intersections with exactly one street used: 2/4 (50.00%)
Intersections with non-trivial schedule: 1/4 (25.00%)
Used streets: 4/5 (80.00%)
Cars that can finish in time: 2/2 (100.00%)
Upper bound of the score: 2,002
----------------------------------------------------------------------
b.txt
Intersections used: 6296/7073 (89.01%)
Intersections with exactly one street used: 4977/7073 (70.37%)
Intersections with non-trivial schedule: 1319/7073 (18.65%)
Used streets: 7964/9102 (87.50%)
Cars that can finish in time: 1000/1000 (100.00%)
Upper bound of the score: 4,576,202
----------------------------------------------------------------------
c.txt
Intersections used: 7660/10000 (76.60%)
Intersections with exactly one street used: 4468/10000 (44.68%)
Intersections with non-trivial schedule: 3192/10000 (31.92%)
Used streets: 11472/35030 (32.75%)
Cars that can finish in time: 1000/1000 (100.00%)
Upper bound of the score: 1,328,389
--------------------------